In [1]:
from google.oauth2.service_account import Credentials
import gspread
import pandas as pd

In [2]:
def prepare_data(metrics):
    # Ensure all numbers are floats, as JSON transmission does not support integers
    # The first column is assumed to be the month and is left as an object/string
    metrics.iloc[:, 1:] = metrics.iloc[:, 1:].astype(float)
    
    # NaNs are not supported, and must be replaced with Python Nones first. That
    # requires a cast to object first.
    metrics = metrics.astype(object).where(metrics.notna(), None)
    
    return metrics

def load(metrics_file):
    metrics = pd.read_csv(metrics_file, sep="\t")
    return metrics.pipe(prepare_data)

def append_last_row(metrics, worksheet):
    last_row = metrics.iloc[-1].to_list()
    
    worksheet.append_row(last_row, value_input_option="USER_ENTERED")
    

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file(
    'lucid-cocoa-399500-86f5d5d1543f.json',
    scopes=scope
)

gc = gspread.authorize(credentials)

## Update editors sheet

This section updates the metrics in the ["Editors and content key product metrics" Google spreadsheet](https://docs.google.com/spreadsheets/d/1wfTtHjQP9Kj0WME15ESJ-4dSMGMpbtY8qOuDVcwZovQ/).

In [3]:
editing_metrics = load("metrics/editing_metrics.tsv")
editing_diversity_metrics = load("metrics/editing_diversity_metrics.tsv")

spreadsheet = gc.open_by_key("1wfTtHjQP9Kj0WME15ESJ-4dSMGMpbtY8qOuDVcwZovQ")

In [4]:
wks1 = spreadsheet.get_worksheet(0)
to_add = editing_metrics.iloc[:,[0, 1, 2, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]]
append_last_row(to_add, wks1)

IndexError: positional indexers are out-of-bounds

In [5]:
wks2 = spreadsheet.get_worksheet(1)
append_last_row(editing_diversity_metrics, wks2)

In [6]:
wks3 = spreadsheet.get_worksheet(2)
mh_metrics = editing_metrics.iloc[:,[0, 6, 7, 8, 9]]
append_last_row(mh_metrics, wks3)

## Update readers sheet

This section updates the metrics in the ["Readers key product metrics" Google spreadsheet](https://docs.google.com/spreadsheets/d/1mK-R8qWzKjSeHMBBek9sJsbecdic9s3r28OIW7QkqrE/).

In [7]:
readers_metrics = load("metrics/readers_metrics.tsv")
readers_diversity_metrics = load("metrics/readers_diversity_metrics.tsv")

spreadsheet = gc.open_by_key("1mK-R8qWzKjSeHMBBek9sJsbecdic9s3r28OIW7QkqrE")

In [9]:
wks1 = spreadsheet.get_worksheet(0)
to_add = readers_metrics.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7]]
append_last_row(to_add, wks1)

In [10]:
wks2 = spreadsheet.get_worksheet(1)
append_last_row(readers_diversity_metrics, wks2)